<a href="https://www.kaggle.com/code/jonbown/formula-1-post-race-summary?scriptVersionId=127927440" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div align='center'>
    <img src="https://www.raconteur.net/wp-content/uploads/2018/04/shutterstock_762930772.jpg"/>
</div>

<style>
@font-face {
  font-family: F1;
  src: url(https://www.formula1.com/etc/designs/fom-website/fonts/F1Regular/Formula1-Regular.ttf);
}

span{
  font-family: F1;
}

a{
  font-family: F1;
}

.nav-pills>li.active>a:focus {
    color: #ffffff;
    background-color: lightgray;
}

.container-fluid, .container-fluid h1 {
    font-family: F1;
    line-height: 1.7;
}

.container-fluid p {
    font-family: F1;
    color:black;
}

h1,h2,h3,h4,h5,h6,p, table {
  font-family: F1;
  color: black
}
</style>

In [1]:
%%html
<style>
@font-face {
  font-family: F1;
  src: url(https://www.formula1.com/etc/designs/fom-website/fonts/F1Regular/Formula1-Regular.ttf);
}

span{
  font-family: F1;
}

a{
  font-family: F1;
}

.nav-pills>li.active>a:focus {
    color: #ffffff;
    background-color: lightgray;
}

.container-fluid, .container-fluid h1 {
    font-family: F1;
    line-height: 1.7;
}

.container-fluid p {
    font-family: F1;
    color:black;
}

h1,h2,h3,h4,h5,h6,p, table {
  font-family: F1;
  color: black
}
</style>

<div style="padding: 15px; 
            color: #FFFFFF;
            margin: 5px;
            font-size: 110%;
            display: block;
            border-radius: 5px;
            border: 5px solid #FF1801;
            background-color: #000000;">
    <center>
        <a id="top"></a>
        <h3 style="margin-bottom: 20px; color: white;">Table of Contents</h3>
    </center>
    <ol>
        <li>Podium</li>
        <li>Full Results</li>
        <li>Qualifying</li>
        <li>Sprint</li>
        <li>Pit Stops</li>
        <li>Lap Summary</li>
        <li>Driver Standings</li>
        <li>Constructor Race Performance</li>
        <li>Constructor Standings</li>
        <li>Sources</li>
    </ol>
</div>


In [2]:
!pip install pyergast

In [3]:
import numpy as np
import pandas as pd
import datetime
from datetime import date
import os
import seaborn as sns
from IPython.display import Markdown as md
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
import plotly.graph_objects as go
pd.set_option('display.precision', 1)
from pyergast import pyergast
import requests
from plotly.offline import init_notebook_mode, iplot

In [4]:
#NOT ALL API ENDPOINTS AVAILABLE IN PYTHON PACKAGE OR WORK CORRECTLY, NEED TO USE MY OWN FOR SOME
def constructor_standings(year=None, race=None):
    """
    Fetch the constructor standings after a specific race in a specific year. Defaults to latest standings
    Parameters
    ----------
    year: int
        An optional parameter that specifies the year to be queried.
    race: int
        An optional parameter that specifies the round of a year to be queried.
    Returns
    -------
    pandas.DataFrame
    Index:
        RangeIndex
    Columns:
        position: int
        positionText: str
        points: int
        wins: int
        constructorID: str
        constructor: str
        nationality: str
    Example
    -------
    >>> pyergast.constructor_standings(1965)
       position positionText points wins    constructorID             name    nationality
    0         1            1     54    6     lotus-climax     Lotus-Climax        British
    1         2            2     45    3              brm              BRM        British
    2         3            3     27    0   brabham-climax   Brabham-Climax        British
    3         4            4     26    0          ferrari          Ferrari        Italian
    4         5            5     14    0    cooper-climax    Cooper-Climax        British
    5         6            6     11    1            honda            Honda       Japanese
    6         7            7      5    0      brabham-brm      Brabham-BRM        British
    7         8            8      2    0        lotus-brm        Lotus-BRM        British
    8         9            9      0    0     brabham-ford     Brabham-Ford        British
    9        10           10      0    0             alfa       Alfa Romeo        Italian
    10       11           11      0    0   lds-alfa_romeo   LDS-Alfa Romeo  South African
    11       12           12      0    0      cooper-ford      Cooper-Ford        British
    12       13           13      0    0       lds-climax       LDS-Climax  South African
    13       14           14      0    0       lotus-ford       Lotus-Ford        British
    14       15           15      0    0               re               RE      Rhodesian
    15       16           16      0    0  cooper-maserati  Cooper-Maserati        British
    """
    if year and race:
        assert year >= 1958, 'Constructor standings only available starting 1958'
        url = 'http://ergast.com/api/f1/{}/{}/constructorStandings.json?limit=1000'.format(year, race)
    elif year:
        assert year >= 1958, 'Constructor standings only available starting 1958'
        url = 'http://ergast.com/api/f1/{}/constructorStandings.json?limit=1000'.format(year, race)
    else:
        url = 'http://ergast.com/api/f1/current/constructorStandings.json?limit=1000'

    r = requests.get(url)
    assert r.status_code == 200, 'Cannot connect to Ergast API. Check your inputs.'
    output = r.json()
    try:
        constructorStandings = output['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']
    except:
        return pd.DataFrame()

    for constructor in constructorStandings:
        constructor['constructorID'] = constructor['Constructor']['constructorId']
        constructor['name'] = constructor['Constructor']['name']
        constructor['nationality'] = constructor['Constructor']['nationality']
        del constructor['Constructor']

    return pd.DataFrame(constructorStandings)



def get_pit_stops(year=None, race=None):
    """
    """
    if year and race:
        url = 'http://ergast.com/api/f1/{}/{}/pitstops.json?limit=1000'.format(year, race)
    r = requests.get(url)

    assert r.status_code == 200, 'Cannot connect to Ergast API'
    drivers = r.json()
    try:
        result = pd.DataFrame(drivers["MRData"]["RaceTable"]["Races"][0]["PitStops"])
    except:
        return pd.DataFrame()

    return result


def get_round(year=None, race=None):
    """
    """
    if year and race:
        url = 'http://ergast.com/api/f1/{}/{}/pitstops.json?limit=1000'.format(year, race)
    r = requests.get(url)

    assert r.status_code == 200, 'Cannot connect to Ergast API'
    drivers = r.json()
    try:
        result = pd.DataFrame(drivers["MRData"]["RaceTable"]["Races"][0]["PitStops"])
    except:
        return pd.DataFrame()

    return result

def get_lap_times(year=None, race=None):
    """
    """
    if year and race:
        url = 'http://ergast.com/api/f1/{}/{}/laps.json?limit=1000'.format(year, race)
    r = requests.get(url)

    assert r.status_code == 200, 'Cannot connect to Ergast API'
    drivers = r.json()
    try:
        result = pd.DataFrame(drivers["MRData"]["RaceTable"]["Races"][0]["Laps"])
    except:
        return pd.DataFrame()
    lap_df = pd.DataFrame()
    for lap in result.number.unique():
        temp_df = pd.DataFrame(result[result['number'] == lap].Timings.iloc[0])
        temp_df['lap'] = lap
        lap_df = pd.concat([lap_df, temp_df], ignore_index=True, axis=0)

    return lap_df

def get_last_round():
    """
    """
    url = 'http://ergast.com/api/f1/current/last/results.json?limit=1000'
    r = requests.get(url)

    assert r.status_code == 200, 'Cannot connect to Ergast API'
    drivers = r.json()
    try:
        result = drivers["MRData"]["RaceTable"]["round"]
    except:
        return pd.DataFrame()
   

    return int(result)


def get_sprint_results(year=None, race=None):
    """
    Get results of sprint, return empty dataframe if no sprint results available
    """
    if year and race:
        url = f'http://ergast.com/api/f1/{year}/{race}/sprint.json?limit=1000'
    r = requests.get(url)

    assert r.status_code == 200, 'Cannot connect to Ergast API'
    drivers = r.json()
    try:
        result = pd.DataFrame(drivers["MRData"]["RaceTable"]["Races"][0]["SprintResults"])
        result['driverID'] = result['Driver'].apply(lambda x: x['driverId'])
        result['Driver'] = result['Driver'].apply(lambda x: x['code'])
        result['constructorId'] = result['Constructor'].apply(lambda x: x['constructorId'])
    except:
        return pd.DataFrame()

    return result

In [5]:
#DATA CLEANING
##INITIAL PARAMETERS
my_date = date.today()
year, week_num, day_of_week = my_date.isocalendar()
race = get_last_round()

#RACE RESULTS
race_result = pyergast.get_race_result()
race_result['points'] = race_result.points.astype(float)
circuits = pyergast.get_circuits()

#CONSTRUCTORS
constructor_overall = constructor_standings(year=year)
const_ov_emp = constructor_overall.empty
if not const_ov_emp:
    constructor_overall['points'] = constructor_overall.points.astype(float)
constructor_results = constructor_standings(year=year, race=race)
const_re_emp = constructor_results.empty
if not const_re_emp:
    constructor_results['points'] = constructor_results.points.astype(float)
constructors = pyergast.get_constructors()

#DRIVERS
drivers = pyergast.get_drivers(year=year)
drivers_2 = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/drivers.csv')
drivers_2 ['Driver'] = drivers_2['code']
drivers_2['driverID'] = drivers_2['driverRef']
drivers['driverID'] = drivers['driverId']
drivers_full = drivers.merge(drivers_2, on='driverID', how='left', suffixes=('_1', '_2'))
driver_standings = pyergast.driver_standings()

#LAPS
lap_times = get_lap_times(year, race)
lap_tim_emp = lap_times.empty
if not lap_tim_emp:
    lap_times['driverID'] = lap_times['driverId']
    lap_times['lap'] = lap_times['lap'].astype(int)
    
#PIT STOPS
pit_stops = get_pit_stops(year, race)
pit_stop_emp = pit_stops.empty
if not pit_stop_emp:
    pit_stops['driverID'] = pit_stops['driverId']
    
#QUALIFYING
qualifying = pyergast.get_qualifying_result()

#SPRINT RESULTS
sprint_results = get_sprint_results(year=year, race=race)
SPRINT_EMPTY = sprint_results.empty

#SCHEDULE
schedule = pyergast.get_schedule()
schedule['round'] = schedule['round'].astype(float)

In [6]:
#Helpers
def get_sec(time_str):
    """Get seconds from time."""
    if (time_str != '\\N' or not time_str):
        m, s = str(time_str).split(':')
        return float(m) * 60 + float(s)
    
def get_sec3(time_str):
    """Get seconds from time."""
    if (time_str != '\\N' or not time_str):
        h, m, s = str(time_str).split(':')
        return float(h)*3600 + float(m) * 60 + float(s)
    
def time_to_seconds(time_str):
    if (time_str != '\\N' or not time_str):
        time_str = str(time_str)
        if time_str == '':
            return None
        parts = time_str.split(':')
        if len(parts) == 1:
            # s.mill format
            return float(parts[0])
        elif len(parts) == 2:
            # minute:seconds.milliseconds format
            minutes = int(parts[0])
            seconds = float(parts[1])
            return minutes * 60 + seconds
        elif len(parts) == 3:
            # hour:minute:seconds.milliseconds format
            hours = int(parts[0])
            minutes = int(parts[1])
            seconds = float(parts[2])
            return hours * 3600 + minutes * 60 + seconds
        else:
            raise ValueError("Invalid time format")

def convert_time_end(row):
    return row['time_date'] + datetime.timedelta(0, row['duration'])

#Colors
color_map = {
        'ALB': '#00A0DE',
        'SAR': '#00A0DE',
        'ZHO': '#A42134',
        'BOT': '#A42134',
        'TSU': '#20394C',
        'DEV': '#20394C',
        'VER': '#000B8D',
        'PER': '#000B8D',
        'HAM': '#00A19B',
        'RUS': '#00A19B',
        'MAG': '#E6002B',
        'HUL': '#E6002B', 
          'ALO': '#002420', 
          'STR': '#002420', 
          'LEC': '#EF1A2D',
          'SAI':'#EF1A2D',
          'OCO': '#2173B8',
          'GAS': '#2173B8',
        'NOR': '#FF8000' ,
        'PIA': '#FF8000'
}


constructor_color_map = {
        'Williams': '#00A0DE',
        'Alfa Romeo': '#A42134',
        'AlphaTauri': '#20394C',
        'Red Bull': '#000B8D',
        'Mercedes': '#00A19B',
        'Haas F1 Team': '#E6002B',
          'Aston Martin': '#002420', 
          'Ferrari': '#EF1A2D',
          'Alpine F1 Team': '#2173B8',
        'McLaren': '#FF8000' 
}

In [7]:
# DATA CLEANING
if not lap_tim_emp:
    lap_times['duration'] = lap_times['time'].apply(get_sec)
    lap_times['red_flag'] = 0
    lap_times.loc[lap_times['duration'] > 300.0, 'red_flag'] = 1


current_circuit = schedule[schedule['round'] == race].circuitID.values[0]
current_race_name = schedule[schedule['round'] == race].circuitName.values[0]
current_race_date = schedule[schedule['round'] == race].date.values[0]


#DRIVER/LAP
if not lap_tim_emp:
    lap_driver = lap_times.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))
    lap_driver['lap'] = lap_driver.lap.astype(float)
    lap_driver = lap_driver[lap_driver['red_flag'] == 0].copy()
    lap_driver_no = lap_driver[(stats.zscore(lap_driver['duration'])) <1]


#RESULTS
num_laps = max(race_result['laps'].astype(int))
merged = race_result.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))
res_race_laps = merged.copy()
res_race_laps['laps'] = 0
reduced = res_race_laps[['laps', 'grid', 'Driver']]
reduced.columns = ['laps', 'position', 'Driver']
reduced_res = pd.concat([reduced, merged[['laps', 'position', 'Driver']]], axis=0)
reduced_res['laps'] = reduced_res['laps'].astype(int)
reduced_res['last_lap'] = 0
reduced_res['Driver'] = reduced_res['Driver'].astype(str)
##Add flag for non-finishers
for driver in color_map.keys():
    temp = reduced_res[reduced_res['Driver'] == driver]
    max_driver_laps = max(temp['laps'])
    if max_driver_laps < num_laps:
        #This driver didn't finish
        reduced_res.loc[(reduced_res['Driver'] == driver) & (reduced_res['laps'] == max_driver_laps), 'last_lap'] = 1

#CONSTRUCTORS
constructor_results = pd.DataFrame()
for i in range(1, race+1):
    if i == 1:
        constructor_results = constructor_standings(year=year, race=i)
        constructor_results['race'] = i
    else:
        temp = constructor_standings(year=year, race=i)
        temp['race'] = i
        constructor_results = pd.concat([constructor_results, temp], axis=0)
constructor_results['points'] = constructor_results['points'].astype(float)



track = circuits[circuits['circuitId'] == current_circuit]
track_lat = track['Latitude'].values[0]
track_lng = track['Longtitude'].values[0]


#PODIIUM
res_race_lap_status_pod = race_result[race_result['position'] != '\\N'].copy()
res_race_lap_status_pod = res_race_lap_status_pod.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))
res_race_lap_status_pod['position'] = res_race_lap_status_pod['position'].astype(int)
res_race_lap_status_pod = res_race_lap_status_pod.sort_values(by='position')


#PIT STOPS
if not pit_stop_emp:
    pit_stops['duration'] = pit_stops['duration'].apply(lambda x: time_to_seconds(x))
    pit_stops['duration'] = pit_stops['duration'].astype(float)
    pit_stops['red_flag'] = (pit_stops['duration'] > 600.0).astype(int)
##PIT STOP DURATIONS
    pit_stops['time_stamp'] = current_race_date + ' ' + pit_stops['time']
    pit_stops['time_date'] = pd.to_datetime(pit_stops['time_stamp'])
    pit_stops['time_end_date'] = pit_stops.apply(lambda x: convert_time_end(x), axis=1)
#MERGE WITH DRIVERS
    driver_group = pd.DataFrame()
    pit_stops = pit_stops.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))
    mean = pit_stops[pit_stops['red_flag'] == 0].groupby(['Driver'])['duration'].mean()
    mini = pit_stops[pit_stops['red_flag'] == 0].groupby(['Driver'])['duration'].min()
    maxi = pit_stops[pit_stops['red_flag'] == 0].groupby(['Driver'])['duration'].max()
    driver_group['mean'] = mean
    driver_group['max'] = maxi
    driver_group['min'] = mini
    driver_group = driver_group.sort_values(by="max")
    driver_group['Driver'] = maxi.index

# QUALIFYING & FINISHING
qualifying = qualifying.fillna('\\N')
qualifying['q1_dur'] = qualifying['Q1'].apply(time_to_seconds)
qualifying['q2_dur'] = qualifying['Q2'].apply(time_to_seconds)
qualifying['q3_dur'] = qualifying['Q3'].apply(time_to_seconds)
qualifying['max_time'] = qualifying.apply(lambda x: max(x['q1_dur'], x['q2_dur'], x['q3_dur']), axis=1)
qualifying['min_time'] = qualifying.apply(lambda x: min(x['q1_dur'], x['q2_dur'], x['q3_dur']), axis=1)
max_time = max(qualifying['max_time']) + 1
min_time = min(qualifying['min_time']) - 1
if max_time > (min_time+15):
    max_time = min_time+15
qualifying = qualifying.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))

#SPRINT
sprint_results['points'] = sprint_results['points'].astype(float)
sprint_results['position'] = sprint_results['position'].astype(int)
sprint_results['grid'] = sprint_results['grid'].astype(int)


num_sprint_laps = max(sprint_results['laps'].astype(int))
#merged = race_result.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))
sp_res_race_laps = sprint_results.copy()
sp_res_race_laps['laps'] = 0
sp_reduced = sp_res_race_laps[['laps', 'grid', 'Driver']]
sp_reduced.columns = ['laps', 'position', 'Driver']
sp_reduced_res = pd.concat([sp_reduced, sprint_results[['laps', 'position', 'Driver']]], axis=0)
sp_reduced_res['laps'] = sp_reduced_res['laps'].astype(int)
sp_reduced_res['last_lap'] = 0
sp_reduced_res['Driver'] = sp_reduced_res['Driver'].astype(str)
for driver in color_map.keys():
    temp = sp_reduced_res[sp_reduced_res['Driver'] == driver]
    max_driver_laps = max(temp['laps'])
    if max_driver_laps < num_sprint_laps:
        #This driver didn't finish
        sp_reduced_res.loc[(sp_reduced_res['Driver'] == driver) & (sp_reduced_res['laps'] == max_driver_laps), 'last_lap'] = 1


In [8]:
display(md(f"# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 38px;'>{current_race_name}, {year} Data Summary</span></h1></div>"))

# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 38px;'>Baku City Circuit, 2023 Data Summary</span></h1></div>

In [9]:
!pip install gmplot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 3.9 MB/s eta 0:00:00


In [10]:
fig = px.scatter_mapbox(track, lat="Latitude", lon="Longtitude",
                        color_discrete_sequence=["fuchsia"], zoom=12, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [11]:
display(md(f"# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Podium</span></h1></div>"))

# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Podium</span></h1></div>

In [12]:
# Styling
def podium(styler):
    styler.set_caption("Podium Results")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    styler.hide_index()
   # styler.format({'points': "{.1f}"})
    return styler


def results_style(styler):
    styler.set_caption("Race Results")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    styler.hide_index()
    return styler

def sprint_results_style(styler):
    styler.set_caption("Sprint Results")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    styler.hide_index()
    return styler

def driver_stand(styler):
    styler.set_caption("Driver Standings")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    styler.hide_index()
    return styler

def const_standings(styler):
    styler.set_caption("Constructor Standings")
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlOrRd")
    styler.hide_index()
    return styler

In [13]:
res_race_lap_status_pod[['Driver', 'grid', 'position', 'points']].iloc[:3].style.pipe(podium)

Driver,grid,position,points
PER,3,1,25.0
VER,2,2,18.0
LEC,1,3,15.0


In [14]:
display(md(f"# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Full Results</span></h1></div>"))

# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Full Results</span></h1></div>

In [15]:
res_race_lap_status_pod[['Driver', 'grid', 'position', 'points', 'status']].style.pipe(results_style)

Driver,grid,position,points,status
PER,3,1,25.0,Finished
VER,2,2,18.0,Finished
LEC,1,3,15.0,Finished
ALO,6,4,12.0,Finished
SAI,4,5,10.0,Finished
HAM,5,6,8.0,Finished
STR,9,7,6.0,Finished
RUS,11,8,5.0,Finished
NOR,7,9,2.0,Finished
TSU,8,10,1.0,Finished


In [16]:
display(md(f"# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Qualifying</span></h1></div>"))

# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Qualifying</span></h1></div>

In [17]:
fig = go.Figure(layout_yaxis_range=[min_time, max_time])
fig.add_trace(go.Bar(
    x=qualifying['Driver'],
    y=qualifying['q1_dur'],
    name='Q1',
    marker_color='red'
))
fig.add_trace(go.Bar(
    x=qualifying['Driver'],
    y=qualifying['q2_dur'],
    name='Q2',
    marker_color='gold'
))
fig.add_trace(go.Bar(
    x=qualifying['Driver'],
    y=qualifying['q3_dur'],
    name='Q3',
    marker_color='orange'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.update_layout(title_text='<b>Qualifying Results</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)
fig.show()

In [18]:
def multi_st_fin_plot(names, addAll = True):
    fig = go.Figure()
    
    lap_driver_sf = reduced_res[(reduced_res['laps'] == 0) | (reduced_res['laps'] == num_laps) | (reduced_res['last_lap'] == 1)].copy()
    lap_driver_sf['position'] = lap_driver_sf['position'].astype(float)
    lap_driver_sf = lap_driver_sf.sort_values(by=['Driver'])

    df = px.data.gapminder().query("continent=='Oceania'")
    fig = px.line(lap_driver_sf, x="laps", y="position", color='Driver', color_discrete_map=color_map, title="Qualifying to Finish", labels={
    "position": "Start Position",
    "laps":"Lap (Start/Finish Only)"
    })
    fig.update_layout(title_text='<b>Qualifying To Finish</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)


    button_all = dict(label = 'All',
                      method = 'update',
                      args = [{'visible': [True]*len(names),
                               'title': 'All',
                               'showlegend':True}])

    def create_layout_button(column):
        return dict(label = column,
                    method = 'update',
                    args = [{'visible': [c == column for c in names],
                             'title': column,
                             'showlegend': True}])

    fig.update_layout(
        updatemenus=[go.layout.Updatemenu(
            active = 0,
            buttons = ([button_all] * addAll) + [create_layout_button(column) for column in names]
            )
        ])
    
    fig.show()
    
mapping = list(color_map.keys())
mapping.sort()
multi_st_fin_plot(mapping)

In [19]:
if not  SPRINT_EMPTY:
    display(md(f"# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Sprint Results</span></h1></div>"))

# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Sprint Results</span></h1></div>

In [20]:
if not SPRINT_EMPTY:
    sprint_results[['Driver', 'grid', 'position', 'points', 'status']].style.pipe(sprint_results_style)

In [21]:
def multi_st_fin_spr_plot(names, addAll = True):
    fig = go.Figure()
    
    lap_driver_sf = sp_reduced_res[(sp_reduced_res['laps'] == 0) | (sp_reduced_res['laps'] == num_sprint_laps) | (sp_reduced_res['last_lap'] == 1)].copy()
    lap_driver_sf['position'] = lap_driver_sf['position'].astype(float)
    lap_driver_sf = lap_driver_sf.sort_values(by=['Driver'])

    df = px.data.gapminder().query("continent=='Oceania'")
    fig = px.line(lap_driver_sf, x="laps", y="position", color='Driver', color_discrete_map=color_map, title="Qualifying to Sprint Finish", labels={
    "position": "Start Position",
    "laps":"Lap (Start/Finish Only)"
    })
    fig.update_layout(title_text='<b>Qualifying To Sprint Finish</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)


    button_all = dict(label = 'All',
                      method = 'update',
                      args = [{'visible': [True]*len(names),
                               'title': 'All',
                               'showlegend':True}])

    def create_layout_button(column):
        return dict(label = column,
                    method = 'update',
                    args = [{'visible': [c == column for c in names],
                             'title': column,
                             'showlegend': True}])

    fig.update_layout(
        updatemenus=[go.layout.Updatemenu(
            active = 0,
            buttons = ([button_all] * addAll) + [create_layout_button(column) for column in names]
            )
        ])
    
    fig.show()
if not SPRINT_EMPTY:
    mapping = list(color_map.keys())
    mapping.sort()
    multi_st_fin_spr_plot(mapping)

In [22]:
display(md(f"# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Pit Stops</span></h1></div>"))

# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Pit Stops</span></h1></div>

In [23]:
if not pit_stop_emp:
    fig = px.timeline(pit_stops[pit_stops['red_flag'] == 0], x_start="time_date", x_end="time_end_date", y="Driver", color='Driver', color_discrete_map=color_map)
    fig.update_layout(showlegend=False)
    fig.update_layout(title_text='<b>Pit Strategy</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)
    fig.show()
else:
    display(md(f"<h1 style='background:#FFF200;border:0; color:black;box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);transform: rotateX(10deg);'><center style='color: black;'>No Pit Data Available From API</center></h1>"))

<h1 style='background:#FFF200;border:0; color:black;box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);transform: rotateX(10deg);'><center style='color: black;'>No Pit Data Available From API</center></h1>

In [24]:
#Red Flags
if not pit_stop_emp and not pit_stops[pit_stops['red_flag'] == 1].empty:
    fig = px.timeline(pit_stops[pit_stops['red_flag'] == 1], x_start="time_date", x_end="time_end_date", y="Driver", color='Driver', color_discrete_map=color_map)
    fig.update_layout(showlegend=False)
    fig.update_layout(title_text='<b>Red Flag Stops</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)
    fig.show()
else:
    display(md(f"<h1 style='background:#FFF200;border:0; color:black;box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);transform: rotateX(10deg);'><center style='color: black;'>No Pit Data Available From API</center></h1>"))

<h1 style='background:#FFF200;border:0; color:black;box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);transform: rotateX(10deg);'><center style='color: black;'>No Pit Data Available From API</center></h1>

In [25]:
if not pit_stop_emp:
    max_pit = max(driver_group['max']) + 1
    min_pit = min(driver_group['min']) - 1

    data = [
    go.Bar(y=driver_group['max'], x=driver_group.Driver, name='Max Pit',  
            #base=0
           ),
    go.Bar(y=driver_group['mean'], x=driver_group.Driver, name='Avg PIt'),
    go.Bar(y=driver_group['min'], x=driver_group.Driver, name='Min PIt')]

    layout = go.Layout(
    barmode='overlay',
    title="Pit Times",
    title_text='<b>Pit Stop Times</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5
    )

    fig = dict(data = data, layout = layout, layout_yaxis_range=[min_pit, max_pit])
    
    iplot(fig, show_link=False)

In [26]:
display(md(f"# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Lap Results</span></h1></div>"))

# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Lap Results</span></h1></div>

In [27]:
if not lap_tim_emp:
    fig = px.box(lap_driver_no, x="duration", y="Driver", color='Driver', points=False, color_discrete_map=color_map, labels={
    "code": "Driver",
    "duration":"Duration (Seconds, Outliers/Red Flags, Removed)"
    })
    fig.update_layout(showlegend=False)
    fig.update_layout(title_text='<b>Lap Times</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)
    fig.show()
else:
        display(md(f"<h1 style='background:#FFF200;border:0; color:black;box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);transform: rotateX(10deg);'><center style='color: black;'>No Lap Data Available From API</center></h1>"))

<h1 style='background:#FFF200;border:0; color:black;box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);transform: rotateX(10deg);'><center style='color: black;'>No Lap Data Available From API</center></h1>

In [28]:
def multi_lap_plot(names, addAll = True):
    fig = go.Figure()

    fig = px.line(lap_driver_no, x="lap", y="duration", color='Driver', labels={
    "duration": "Lap Time (In seconds, Outliers/Red Flags Removed)",
    "lap": "Lap"
}, title='Lap Times', color_discrete_map=color_map)
    fig.update_layout(title_text='<b>Lap Times</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)

    button_all = dict(label = 'All',
                      method = 'update',
                      args = [{'visible': [True]*len(names),
                               'title': 'All',
                               'showlegend':True}])

    def create_layout_button(column):
        return dict(label = column,
                    method = 'update',
                    args = [{'visible': [c == column for c in names],
                             'title': column,
                             'showlegend': True}])

    fig.update_layout(
        updatemenus=[go.layout.Updatemenu(
            active = 0,
            buttons = ([button_all] * addAll) + [create_layout_button(column) for column in names]
            )
        ])
    
    fig.show()
    
    
    
def multi_pos_plot(names, addAll = True):
    fig = go.Figure()

    fig = px.line(lap_driver, x="lap", y="position", color='Driver', labels={
    "position":  "Position",
    "lap": "Lap"
}, color_discrete_map=color_map)
    fig.update_layout(title_text='<b>Position By Lap</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)

    button_all = dict(label = 'All',
                      method = 'update',
                      args = [{'visible': [True]*len(names),
                               'title': 'All',
                               'showlegend':True}])

    def create_layout_button(column):
        return dict(label = column,
                    method = 'update',
                    args = [{'visible': [c == column for c in names],
                             'title': column,
                             'showlegend': True}])

    fig.update_layout(
        updatemenus=[go.layout.Updatemenu(
            active = 0,
            buttons = ([button_all] * addAll) + [create_layout_button(column) for column in names]
            )
        ])
    
    fig.show()


In [29]:
if not lap_tim_emp:
    multi_lap_plot(mapping)

In [30]:
if not lap_tim_emp:
    multi_pos_plot(mapping)

In [31]:
display(md(f"# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Driver Overall Standings</span></h1></div>"))

# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Driver Overall Standings</span></h1></div>

In [32]:
drive_stand = driver_standings.merge(drivers_full, on='driverID', how='left', suffixes=('_1', '_2'))
drive_stand['points'] = drive_stand.points.astype(float)
drive_stand_race  = drive_stand .sort_values(by='points', ascending=False)
drive_stand_race[['Driver', 'points']].style.pipe(driver_stand)

Driver,points
VER,69.0
PER,54.0
ALO,45.0
HAM,38.0
SAI,20.0
STR,20.0
RUS,18.0
NOR,8.0
HUL,6.0
LEC,6.0


In [33]:
display(md(f"# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Constructors Race Performance</span></h1></div>"))

# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Constructors Race Performance</span></h1></div>

In [34]:
fig = go.Figure()

fig = px.line(constructor_results, x="race", y="points", color='name', labels={
    "points": "Points",
    "race": "Race"
}, title='Lap Times', color_discrete_map=constructor_color_map)
fig.update_layout(title_text='<b>Constructor Championship Timeline</b>', titlefont = dict(
         family = 'Arial, sans-serif',
         size = 30
      ), title_x=0.5)
fig.update_layout(xaxis_range=[1,24])

In [35]:
display(md(f"# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Constructor Overall Standings</span></h1></div>"))

# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Constructor Overall Standings</span></h1></div>

In [36]:
const_stand_race = constructor_overall.sort_values(by='points', ascending=False)
const_stand_race[['name', 'points', 'wins']].style.pipe(const_standings)

name,points,wins
Red Bull,123.0,3
Aston Martin,65.0,0
Mercedes,56.0,0
Ferrari,26.0,0
McLaren,12.0,0
Alpine F1 Team,8.0,0
Haas F1 Team,7.0,0
Alfa Romeo,6.0,0
AlphaTauri,1.0,0
Williams,1.0,0


In [37]:
max_round = max(schedule['round'])
if race < max_round:
    next_race = schedule[schedule['round'] == (race+1)]
    next_race_name = next_race['raceName'].values[0]
    next_race_date = next_race['date'].values[0]
    display(md(f"<h1 style='background:#FF1801;border:0; color:black;box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);transform: rotateX(10deg);'><center style='color: white;'>Next Up: {next_race_name}, {next_race_date}</center></h1>"))
else:
    display(md(f"<h1 style='background:#FF1801;border:0; color:black;box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);transform: rotateX(10deg);'><center style='color: white;'>Next Up: {year+1} Data Summary</center></h1>"))


<h1 style='background:#FF1801;border:0; color:black;box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);transform: rotateX(10deg);'><center style='color: white;'>Next Up: Miami Grand Prix, 2023-05-07</center></h1>

<h1 style='background:#FF1801;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: black;'>Thank You</center></h1>

### Please upvote if you liked this notebook and comment with your feedback! Thank you so much for supporting my work. 

In [38]:
display(md(f"# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Acknowledgements</span></h1></div>"))

# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Acknowledgements</span></h1></div>

- Shoutout to @VOPANI for creating this fantastic, F1 dataset
- The F1 Developer API http://ergast.com/mrd/
- The python package ergast API wrapper, [[pyergast](https://github.com/weiranyu/pyErgast)]
- Very inspired by this [[notebook](https://www.kaggle.com/code/ekrembayar/formula-1-70th-anniversary)] on the same dataset

In [39]:
display(md(f"# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Race Schedule</span></h1></div>"))

# <div style='background:#FF1801; border:0; color:black; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75); transform: rotateX(10deg);'><h1 style='padding: 10px; margin: 0; text-align:center;'><span style='font-size: 30px;'>Race Schedule</span></h1></div>

<div class="row" style='float:left' align='center'>
<div class="column" align = 'center' style='float:left; max-width:50%;'> 
    <img src="https://cdn-1.motorsport.com/images/mgl/Y99JQRbY/s800/red-bull-racing-logo-1.jpg" width=70>
    <img src="https://media.formula1.com/content/dam/fom-website/2018-redesign-assets/team%20logos/alpine.jpg" width=70>
    <img src="https://cdn-8.motorsport.com/images/mgl/YEQJgyLY/s800/mclaren-f1-team-logo-1.jpg" width=60>
    <img src="https://assets.turbologo.com/blog/en/2019/10/19084952/ferrari-logo-illustration-958x575.jpg" width=70 />
      <img src="https://cdn.sanity.io/images/fnx611yr/production/ff454bb2b4adb541b66477adcb49088f8de0fb4a-2400x2400.jpg?rect=0,572,2400,1256&w=1200&h=628&auto=format"/ width=70>
    <img src="https://cdn-9.motorsport.com/images/mgl/0RrzmDo0/s800/mercedes-f1-logo-1.jpg" width=70/>
    <img src="https://diginomica.com/sites/default/files/images/2019-05/Haas-F1-logo.jpg" width=70>
    <img src="https://d3cm515ijfiu6w.cloudfront.net/wp-content/uploads/2021/01/01082241/Aston-Martin.jpg" width=70>
    <img src="https://cdn-2.motorsport.com/images/mgl/Y99JQR8Y/s8/scuderia-alphatauri-f1-logo-1.jpg" width=70>
      <img src="https://www.formula1.com/content/dam/fom-website/manual/teams/Sauber/Alfa_Romeo_Racing_logo.jpg"/ width=70>
</div>
 <div class="column" style='float:left; max-width:50%;'> 
    <img src="https://pbs.twimg.com/media/FmrBRv9XwAAFv18.jpg:large" />
</div>
 </div>